In [2]:
import pandas as pd
from sklearn.model_selection import ParameterGrid
from neuralprophet import NeuralProphet, set_log_level, set_random_seed
from src.model.features import add_stock_price_feature

# Disable logging messages unless there is an error
set_log_level("ERROR")

df = pd.read_csv('data/stocks/2330_stock_data_0317.csv', parse_dates=['ds'])
df = add_stock_price_feature(df)
# plt = df.plot(x="ds", y="y", figsize=(15, 5))
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 566 entries, 4 to 569
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Unnamed: 0                566 non-null    int64         
 1   ds                        566 non-null    datetime64[ns]
 2   open_price                566 non-null    float64       
 3   high_price                566 non-null    float64       
 4   low_price                 566 non-null    float64       
 5   y                         566 non-null    float64       
 6   volume                    566 non-null    int64         
 7   foreign                   566 non-null    int64         
 8   investment                566 non-null    int64         
 9   dealer                    566 non-null    int64         
 10  ratio_over_400_shares     566 non-null    float64       
 11  shareholders_400_to_600   566 non-null    int64         
 12  shareholders_600_to_80

In [3]:
from src.model.utils import val_mape

param_grid = {
    'foreign': [0, 5, 10],
    'investment': [0, 5, 10],
    'dealer': [0, 5, 10]
}

results = []
# Iterate over each combination of hyperparameters
for params in ParameterGrid(param_grid):
    # Initialize the NeuralProphet model with current hyperparameters
    print([params])
    m = NeuralProphet(
        yearly_seasonality=False,
        weekly_seasonality=True,
        n_lags=3,
    )
    m = m.add_country_holidays("TW")
    
    lag_regs = (
        # ('volume', 8),
        # ('high_low_diff', 10),
        ('foreign', params['foreign']),
        ('investment', params['investment']),
        ('dealer', params['dealer'])
    )
    columes = ['ds', 'y']
    for col, lag in lag_regs:
        if lag > 0:
            m.add_lagged_regressor(col, n_lags=lag)
            columes.append(col)
    
    df_train, df_val = m.split_df(df[columes], valid_p=0.2)
    set_random_seed(0)
    metrics = m.fit(df_train, validation_df=df_val)
    
    # Create a new dataframe reaching 365 into the future for our forecast, n_historic_predictions also shows historic data
    df_future = m.make_future_dataframe(df[columes], n_historic_predictions=True, periods=7)

    # Predict the future
    forecast = m.predict(df_future)
    
    rmse = metrics.iloc[-1]['RMSE_val']
    mape = val_mape(df_val, forecast)
    results.append({**params, 'RMSE': rmse, 'MAPE': mape})

[{'dealer': 0, 'foreign': 0, 'investment': 0}]
Training: |          | 0/? [00:00<?, ?it/s]

WARNING - (py.warnings._showwarnmsg) - c:\Users\nick\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\utilities\data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 32. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.

WARNING - (py.warnings._showwarnmsg) - c:\Users\nick\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\utilities\data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 11. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.

Finding best initial lr: 100%|██████████| 219/219 [00:00<00:00, 266.02it/s]


Training: |          | 0/? [00:09<?, ?it/s, v_num=1440, MAE_val=14.80, RMSE_val=18.10, Loss_val=0.00162, RegLoss_val=0.000, train_loss=0.000691, reg_loss=0.000, MAE=8.240, RMSE=11.90, Loss=0.000666, RegLoss=0.000]
Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 223.14it/s]
[{'dealer': 0, 'foreign': 0, 'investment': 5}]
Training: |          | 0/? [00:00<?, ?it/s]

WARNING - (py.warnings._showwarnmsg) - c:\Users\nick\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\utilities\data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 9. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.

Finding best initial lr: 100%|██████████| 219/219 [00:00<00:00, 272.53it/s]


Training: |          | 0/? [00:09<?, ?it/s, v_num=1441, MAE_val=14.50, RMSE_val=17.70, Loss_val=0.00157, RegLoss_val=0.000, train_loss=0.00068, reg_loss=0.000, MAE=8.430, RMSE=11.90, Loss=0.000678, RegLoss=0.000] 
Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 193.15it/s]
[{'dealer': 0, 'foreign': 0, 'investment': 10}]
Training: |          | 0/? [00:00<?, ?it/s]

...

Training: |          | 0/? [00:09<?, ?it/s, v_num=1464, MAE_val=16.20, RMSE_val=19.80, Loss_val=0.00196, RegLoss_val=0.000, train_loss=0.00064, reg_loss=0.000, MAE=8.130, RMSE=11.80, Loss=0.000629, RegLoss=0.000] 
Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 191.31it/s]
[{'dealer': 10, 'foreign': 10, 'investment': 5}]
Training: |          | 0/? [00:00<?, ?it/s]

Finding best initial lr: 100%|██████████| 219/219 [00:00<00:00, 271.90it/s]


Training: |          | 0/? [00:10<?, ?it/s, v_num=1465, MAE_val=16.20, RMSE_val=19.70, Loss_val=0.00194, RegLoss_val=0.000, train_loss=0.000645, reg_loss=0.000, MAE=8.550, RMSE=12.10, Loss=0.000696, RegLoss=0.000]
Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 250.03it/s]
[{'dealer': 10, 'foreign': 10, 'investment': 10}]
Training: |          | 0/? [00:00<?, ?it/s]

Finding best initial lr: 100%|██████████| 219/219 [00:00<00:00, 267.52it/s]


Training: |          | 0/? [00:10<?, ?it/s, v_num=1466, MAE_val=16.30, RMSE_val=19.80, Loss_val=0.00195, RegLoss_val=0.000, train_loss=0.000644, reg_loss=0.000, MAE=8.230, RMSE=11.60, Loss=0.000624, RegLoss=0.000]
Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 249.94it/s]


In [4]:
results_df = pd.DataFrame(results)
results_df.to_csv('params.csv')
results_df

,dealer,foreign,investment,RMSE,MAPE
0,0,0,0,18.057192,0.013987
1,0,0,5,17.749821,0.013916
2,0,0,10,18.220335,0.014399
3,0,5,0,18.484234,0.014421
4,0,5,5,18.236444,0.014255
5,0,5,10,18.934738,0.014747
6,0,10,0,18.750698,0.014567
7,0,10,5,18.919357,0.014637
8,0,10,10,18.647554,0.014579
9,5,0,0,19.248236,0.015059


In [5]:
results_df.sort_values(by="RMSE").head(10)

,dealer,foreign,investment,RMSE,MAPE
1,0,0,5,17.749821,0.013916
0,0,0,0,18.057192,0.013987
2,0,0,10,18.220335,0.014399
4,0,5,5,18.236444,0.014255
3,0,5,0,18.484234,0.014421
8,0,10,10,18.647554,0.014579
6,0,10,0,18.750698,0.014567
7,0,10,5,18.919357,0.014637
5,0,5,10,18.934738,0.014747
9,5,0,0,19.248236,0.015059


In [6]:
results_df.sort_values(by="MAPE").head(10)

,dealer,foreign,investment,RMSE,MAPE
1,0,0,5,17.749821,0.013916
0,0,0,0,18.057192,0.013987
4,0,5,5,18.236444,0.014255
2,0,0,10,18.220335,0.014399
3,0,5,0,18.484234,0.014421
6,0,10,0,18.750698,0.014567
8,0,10,10,18.647554,0.014579
7,0,10,5,18.919357,0.014637
5,0,5,10,18.934738,0.014747
23,10,5,10,19.386820,0.014985
